In [93]:
import pandas as pd
import numpy as np
import scipy

m1_articles = pd.read_csv('articles_sample.csv')
m1_customers = pd.read_csv('customers_sample.csv')
m1_trans = pd.read_csv('transactions_sample.csv')

m1_trans['purchased'] = 1

df = m1_trans.pivot_table(index = 'customer_id', columns ='article_id', values = 'purchased').fillna(0)
df

article_id,108775015,108775044,108775051,110065001,110065002,110065011,111565001,111586001,111593001,111609001,...,946748003,946748004,946764002,946764003,947060001,947509001,949551001,949551002,952267001,952938001
customer_id,,,,,,,,,,,,,,,,,,,,,
000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a7d9c965efcc9187cbf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001270f9c6b9e0fe78121ca9d9071f4b9078da72258a7aed35bdaf125a65b6d0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0019c05146d30111b7003700c712f9354fb62f9b87e53b1c9cc9f0d22fb5c62b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001e4019bd815f027c0b277bb75a74c9b977ad4970576d9dd98aedbe670f2cff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00227494dd4e87da02bb1ab4afc38f13f2e11c6517b1bbcdca56f212a9c470ab,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffd8a6ce04a08854dc847409f7b9f1506f4509482ef73c89bf98421859a543b2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ffdae68f247ef0c78da5bf053dd68b7b4e9aeeec8135bb75ae123a60fb25671b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ffe44295c63a13498687134a9e6ee5c57e08d84bfffa0d810821c1181b8873e1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
# Use SVD to retrieve the recommendation predication matrix

customers_ids = df.index.values.tolist()

# users_articles_mean is the average value of (sum of unique item purchased over total numbers of articles)
users_articles_mean = df.sum(axis=1) / len(df.columns)
users_articles_mean_np = users_articles_mean.to_numpy()

# df_subtract_mean records each entry after the substraction of average value from users_articles_mean_np
df_subtract_mean = df.to_numpy() - users_articles_mean_np.reshape(-1, 1)

# compute the largest or smallest k singular values and corresponding singular vectors of a sparse matrix df_subtract_mean
# left_singular_vec_U represents the relationship between users and latent factors
# singular_values_sigma describes the strength of each latent factor
# right_singular_vec_Vt indicates the similarity between items and latent factors

# latent factors are the characteristics of the items
left_singular_vec_U, singular_values_sigma, right_singular_vec_Vt = scipy.sparse.linalg.svds(df_subtract_mean)

recommendation_mat = pd.DataFrame(np.dot(np.dot(left_singular_vec_U, np.diag(singular_values_sigma)), right_singular_vec_Vt) +
                        users_articles_mean_np.reshape(-1, 1),
                        columns = df.columns)
recommendation_mat

article_id,108775015,108775044,108775051,110065001,110065002,110065011,111565001,111586001,111593001,111609001,...,946748003,946748004,946764002,946764003,947060001,947509001,949551001,949551002,952267001,952938001
0,0.000274,0.000309,0.000071,0.000109,0.000077,0.000096,0.000171,0.000337,0.000272,0.000116,...,0.000032,0.000087,0.000067,6.586862e-05,6.532147e-05,0.000073,0.000068,0.000063,0.000068,0.000067
1,0.000365,0.000350,0.000010,0.000060,0.000021,0.000040,0.000171,0.000480,0.000351,0.000054,...,-0.000046,0.000048,0.000002,1.366513e-07,-8.784372e-07,0.000018,0.000002,-0.000001,0.000003,0.000002
2,0.008018,0.007460,0.000075,0.001160,0.000413,0.000852,0.003362,0.010407,0.007733,0.001201,...,0.003131,0.000772,-0.000089,-1.525642e-04,-1.701676e-04,0.000518,-0.000128,-0.000118,-0.000100,-0.000108
3,0.000218,0.000132,0.000013,0.000028,0.000029,0.000032,0.000057,0.000256,0.000187,0.000059,...,-0.000088,-0.000012,0.000009,7.580106e-06,7.352921e-06,0.000028,0.000008,0.000018,0.000010,0.000007
4,0.015165,0.017771,0.000966,0.003855,0.002236,0.004196,0.004721,0.015673,0.013674,0.008087,...,-0.006692,-0.002099,0.000816,5.907211e-04,6.026833e-04,0.001792,0.000869,0.001099,0.000877,0.000473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13632,0.003808,0.003071,0.000067,0.000482,0.000255,0.000395,0.001352,0.004844,0.003528,0.000657,...,0.000459,0.000172,-0.000015,-4.167870e-05,-5.067009e-05,0.000309,-0.000033,0.000027,-0.000009,-0.000027
13633,0.000247,0.000167,0.000034,0.000051,0.000047,0.000049,0.000096,0.000299,0.000220,0.000066,...,-0.000092,0.000027,0.000029,2.824568e-05,2.783913e-05,0.000044,0.000029,0.000036,0.000030,0.000028
13634,0.007362,0.006282,0.000067,0.000937,0.000321,0.000552,0.003262,0.010008,0.007176,0.000651,...,0.002985,0.000995,-0.000096,-1.346539e-04,-1.561199e-04,0.000414,-0.000131,-0.000127,-0.000103,-0.000079
13635,0.002359,0.001808,0.000010,0.000262,0.000220,0.000384,0.000363,0.002447,0.001934,0.000791,...,-0.000903,-0.000399,-0.000037,-6.920058e-05,-6.920929e-05,0.000261,-0.000051,0.000060,-0.000029,-0.000080


In [95]:
df['index'] = df.index
df.index = np.arange(1, len(df) + 1)

In [96]:
def recommend(userID):
    # lookup user's id
    userID_alt = df.loc[df['index'] == userID].index[0]
    predicted_articles = recommendation_mat.iloc[userID_alt].sort_values(ascending=False)[:10].to_frame()
    predicted_articles['article_id'] = predicted_articles.index
    predicted_articles.index = np.arange(0, len(predicted_articles))
    results = []
    for i in range(0, 10):
        temp = m1_articles.loc[m1_articles['article_id']==predicted_articles.iloc[i]['article_id']].to_dict()
        results.append(temp)
    for article in results:
        for key in article.keys():
            article[key] = article[key][list(article[key].keys())[0]]
    return pd.DataFrame().from_dict(results)


In [97]:
recommend('000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a7d9c965efcc9187cbf')


C:\Users\ltxom\AppData\Local\Temp\ipykernel_23536\1925151747.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  predicted_articles = recommendation_mat.iloc[userID_alt].sort_values(ascending=False)[:10].to_frame()


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,610776002,610776,Tilly (1),255,T-shirt,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,T-shirt in lightweight jersey with a rounded h...
1,372860001,372860,7p Basic Shaftless,302,Socks,Socks & Tights,1010016,Solid,9,Black,...,Shopbasket Socks,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Fine-knit trainer socks in a soft cotton blend.
2,610776001,610776,Tilly (1),255,T-shirt,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,T-shirt in lightweight jersey with a rounded h...
3,372860002,372860,7p Basic Shaftless,302,Socks,Socks & Tights,1010016,Solid,10,White,...,Shopbasket Socks,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Fine-knit trainer socks in a soft cotton blend.
4,720125001,720125,SUPREME RW tights,273,Leggings/Tights,Garment Lower body,1010016,Solid,9,Black,...,Ladies Sport Bottoms,S,Sport,26,Sport,5,Ladies H&M Sport,1005,Jersey Fancy,Sports tights in fast-drying functional fabric...
5,759871002,759871,Tilda tank,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,EQ Divided Basics,D,Divided,2,Divided,80,Divided Complements Other,1002,Jersey Basic,"Cropped, fitted top in cotton jersey with narr..."
6,351484002,351484,Lazer Razer Brief,59,Swimwear bottom,Swimwear,1010016,Solid,9,Black,...,Swimwear,B,Lingeries/Tights,1,Ladieswear,60,"Womens Swimwear, beachwear",1018,Swimwear,"Fully lined bikini bottoms with a mid waist, m..."
7,751471001,751471,Pluto RW slacks (1),272,Trousers,Garment Lower body,1010016,Solid,9,Black,...,Trouser,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1009,Trousers,Ankle-length cigarette trousers in a stretch w...
8,749699001,749699,Chestnut strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,11,Womens Tailoring,1005,Jersey Fancy,Top in viscose jersey with a V-neck front and ...
9,688537004,688537,Simple as that Cheeky Tanga,59,Swimwear bottom,Swimwear,1010026,Other structure,9,Black,...,Swimwear,B,Lingeries/Tights,1,Ladieswear,60,"Womens Swimwear, beachwear",1018,Swimwear,Fully lined bikini bottoms with a mid waist an...
